### Install dependencies

In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate

### Create and save the datasets

In [1]:
%load_ext autoreload
%autoreload 1

import datasets
from transformers import GPT2TokenizerFast
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import TrainingArguments, Trainer
from transformers import GPT2LMHeadModel
import os
from transformers import GPT2TokenizerFast, GPT2LMHeadModel, TrainingArguments, Trainer, default_data_collator
import datasets
import torch
from itertools import chain

if not os.path.isdir("trained_models"):
    os.mkdir("trained_models")

device = "cuda" if torch.cuda.is_available() else "cpu"
model_id = "gpt2"

dataset = datasets.load_dataset(
    "ucberkeley-dlab/measuring-hate-speech",
    "binary"
)["train"]

dataset = dataset.filter(lambda example: example["target_race"] == True)

hate_speech_dataset = dataset.filter(lambda example: example["hate_speech_score"] > 0.5)
hate_speech_dataset.to_json("hate_speech_dataset.json")

supportive_speech_dataset = dataset.filter(lambda example: example["hate_speech_score"] < -1)
supportive_speech_dataset.to_json("supportive_speech_dataset.json")

/home/andreib/miniconda3/envs/bias_measure/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset parquet (/home/andreib/.cache/huggingface/datasets/ucberkeley-dlab___parquet/ucberkeley-dlab--measuring-hate-speech-c32713cabe528196/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 1/1 [00:00<00:00, 22.89it/s]
Loading cached processed dataset at /home/andreib/.cache/huggingface/datasets/ucberkeley-dlab___parquet/ucberkeley-dlab--measuring-hate-speech-c32713cabe528196/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-f655188cf9fee53d.arrow
Loading cached processed dataset at /home/andreib/.cache/huggingface/datasets/ucberkeley-dlab___parquet/ucberkeley-dlab--measuring-hate-speech-c32713cabe528196/0.0.0/2a3b91fbd88a2c90d1dbbb32b

21585
15578


### Train the models

In [9]:
!bash train_models.sh hate

05/01/2023 10:29:35 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
05/01/2023 10:29:35 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_ste

In [10]:
!bash train_models.sh supportive

05/01/2023 12:06:29 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
05/01/2023 12:06:29 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_ste